In [15]:
!which python; python -V;
from astropy import units as u
import time

from poliastro.bodies import Sun, Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune
from poliastro.twobody import Orbit
from poliastro.ephem import Ephem
from poliastro.util import time_range

from perylune.orbit_tools import *
from perylune.mpc import *

/home/thomson/devel/perylune/venv/bin/python
Python 3.8.5


In [2]:
# First step is to load the data. parse_txt() will load the orbital elements from text file in MPC format.
# This will be returned as an array. Each entry in that array has a list of 8 parameters or so.
elements = parse_txt("../data/mpc/mpcorb_extended.dat", limit=2000000, skip="------------------")

Read 1028498 line(s) from NEA.txt, parsing up to 2000000 line(s)
Loaded 50000 entries so far.
Loaded 100000 entries so far.
Loaded 150000 entries so far.
Loaded 200000 entries so far.
Loaded 250000 entries so far.
Loaded 300000 entries so far.
Loaded 350000 entries so far.
Loaded 400000 entries so far.
Loaded 450000 entries so far.
Loaded 500000 entries so far.
Loaded 550000 entries so far.
Loaded 600000 entries so far.
Loaded 650000 entries so far.
Loaded 700000 entries so far.
Loaded 750000 entries so far.
Loaded 800000 entries so far.
Loaded 850000 entries so far.
Loaded 900000 entries so far.
Loaded 950000 entries so far.
Loaded 1000000 entries so far.


In [3]:
# Now filter those with highest specific parameters of interest. Each line has the format returned by parse_line function
# in perylune/mpc.py file: (name, epoch, mean_anomaly, argp, raan, inc, ecc, mean_motion, a, flags)
# For example name is e[0].
lowest_dv_names = [ "2010 FN", "2018 UC", "2016 JA", "2020 SO", "2019 YB4", "2019 AU6", "2000 SG344", "2010 JK1", "2016 WQ3", "2013 BS45" ]
lowest_dv_elements = find_objects(elements, lambda e : e[0] in lowest_dv_names)

high_inc_elements = find_objects(elements, lambda e : e[5] > 45*u.deg) # inclination greater than 45 degrees
high_ecc_elements = find_objects(elements, lambda e : e[6] > 0.7*u.one) # eccentricity over 6

print("Selected %d out of %d objects of interest (criteria: lowest delta-v)" % (len(lowest_dv_elements), len(elements)))
print("Selected %d out of %d objects of interest (criteria: inc > 45 deg)" % (len(high_inc_elements), len(elements)))
print("Selected %d out of %d objects of interest (criteria: ecc > 0.7)" % (len(high_ecc_elements), len(elements)))

10 out of 1028453 elements matched critera.
733 out of 1028453 elements matched critera.
1879 out of 1028453 elements matched critera.
Selected 10 out of 1028453 objects of interest (criteria: lowest delta-v)
Selected 733 out of 1028453 objects of interest (criteria: inc > 45 deg)
Selected 1879 out of 1028453 objects of interest (criteria: ecc > 0.7)


In [17]:

# Ideal, read from the porkchop
start = Time("2020-07-15 11:50", scale="utc").tdb
date_arrival = Time("2021-01-20 12:00", scale="utc").tdb

period = time_range(start, end=date_arrival)

mercury = Ephem.from_body(Mercury, time_range(start, end=start + 0.241*u.year))
venus = Ephem.from_body(  Venus,   time_range(start, end=start + 0.615*u.year))
earth = Ephem.from_body(  Earth,   time_range(start, end=start + 1.00*u.year))
mars = Ephem.from_body(   Mars,    time_range(start, end=start + 1.881*u.year))
jupiter = Ephem.from_body(Jupiter, time_range(start, end=start + 11.86*u.year))
saturn = Ephem.from_body( Saturn,  time_range(start, end=start + 29.42*u.year))
uranus = Ephem.from_body( Uranus,  time_range(start, end=start + 84.02*u.year))
neptune = Ephem.from_body(Neptune, time_range(start, end=start + 164.8  *u.year))

from poliastro.plotting import OrbitPlotter3D
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(mercury, start, label="Mercury")
plotter.plot_ephem(venus, start, label="Venus")
plotter.plot_ephem(earth, start, label="Earth")
plotter.plot_ephem(mars, start, label="Mars")
plotter.plot_ephem(jupiter, start, label="Jupiter")
plotter.plot_ephem(saturn, start, label="Saturn")
plotter.plot_ephem(uranus, start, label="Uranus")
plotter.plot_ephem(neptune, start, label="Neptune")

plotter._layout.autosize = False
plotter._layout.width = 1200
plotter._layout.height = 800
plotter._layout.margin=dict(l=10, r=10, b=10, t=10, pad=4 )
plotter.show()


In [ ]:
len()

In [56]:
plotter2 = OrbitPlotter3D()
plotter2.set_attractor(Sun)

plotter2.plot_ephem(mercury, start, label="Mercury")
plotter2.plot_ephem(venus, start, label="Venus")
plotter2.plot_ephem(earth, start, label="Earth")
plotter2.plot_ephem(mars, start, label="Mars")

#name, epoch, mean_anomaly, argp, raan, inc, ecc, mean_motion, a, flags, epoch)
from pprint import pprint

for e in lowest_dv_elements:
    o = Orbit.from_classical(Sun, a=e[8], ecc=e[6], inc=e[5], raan=e[4], argp=e[3], nu=e[2], epoch=e[1], plane=Planes.EARTH_ECLIPTIC)
    x = plotter2.plot(o, label=e[0])
    x.data[-3].name = "minut3"
    x.data[-2].name = "minut2"
    #print(x.data[-2].name)
    break



plotter2._layout.autosize = False
plotter2._layout.width = 1200
plotter2._layout.height = 800
plotter2._layout.margin=dict(l=10, r=10, b=10, t=10, pad=4 )
plotter2.show()

SyntaxError: invalid syntax (<ipython-input-56-60df0e3ff4e8>, line 16)

In [39]:
# Now let's find some weird asteroids. those with highly eccentric orbits and still reasonably close to Earth.
weirdos2 = find_objects(elements, lambda e : e[6] > 0.9*u.one and e[8] < 2*u.au) # eccentricity over 6

print("Selected %d out of %d objects of interest (criteria: highly eccentric and close to Earth)" % (len(weirdos2), len(elements)))

16 out of 1028453 elements matched critera.
Selected 16 out of 1028453 objects of interest (criteria: highly eccentric and close to Earth)
